In [2]:
import pandas as pd 
import numpy as np 
import pandas_datareader as web
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go 
from plotly.offline import iplot
plt.style.use('fivethirtyeight')

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
# Creating stock dataframe 
stock = web.DataReader('AAPL', data_source='yahoo', start = '2009-05-03', end = '2019-05-03')
stock = stock.reset_index()
stock

,Date,High,Low,Open,Close,Volume,Adj Close
0,2009-05-04,18.892857,18.240000,18.320000,18.867144,152339600.0,16.333237
1,2009-05-05,18.980000,18.731428,18.821428,18.958570,99563800.0,16.412386
2,2009-05-06,19.071428,18.602858,19.047142,18.928572,118384700.0,16.386417
3,2009-05-07,18.912857,18.271429,18.904285,18.437143,132944000.0,15.960989
4,2009-05-08,18.747143,18.037144,18.434286,18.455715,116991000.0,15.977063
...,...,...,...,...,...,...,...
2513,2019-04-29,205.970001,203.860001,204.399994,204.610001,22204700.0,201.421219
2514,2019-04-30,203.399994,199.110001,203.059998,200.669998,46534900.0,197.542618
2515,2019-05-01,215.309998,209.229996,209.880005,210.520004,64827300.0,207.239120
2516,2019-05-02,212.649994,208.130005,209.839996,209.149994,31996300.0,205.890457


In [4]:
trace = go.Scatter(x = stock['Date'], y = stock['Close'], marker = {'color':"#3f8fcc"})
layout = go.Layout(title = {'text':'AAPL 10Y <b>Close price</b>','x':0.5,'y':0.88,'font':{'color':'#404040'}},plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.yaxis.title.text = "AAPL close price"
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [5]:
# constructing dataframe 
data = pd.DataFrame()
data['Date'] = stock.Date

# constructing 12 day EMA 
data['ema12'] = stock.Close.ewm(span=12, adjust=False).mean()

# constructing 26 day EMA
data['ema26'] = stock.Close.ewm(span=26, adjust=False).mean()

# constructing MACD line
data['MACD'] = data['ema12'] - data['ema26'] 

# constructing signal line
data['signal line'] = data.MACD.ewm(span=9, adjust=False).mean()

# constructing MACD histogram
data['hist'] = data['MACD'] - data['signal line']
data['positive'] = data['hist'][data['hist'] > 0]
data['negative'] = data['hist'][data['hist'] < 0]

In [6]:
# visualizing macd, 12day ema, 26 day ema, and signal line 

ema12 = go.Scatter(x = data.Date, y = data.ema12, name="AAPL EMA 12 day", marker = {'color':'#3fcc7c'})
ema26 = go.Scatter(x = data.Date, y = data.ema26, name="AAPL EMA 26 day", marker = {'color':'#3f8fcc'})
layout = go.Layout(title = {'text':'Construction of <b>Signal Line</b>', 'x':0.5,'y':0.88, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.yaxis.title.text = "AAPL Close Price"
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.xaxis.range = ['2013-05-11','2014-03-11']
layout.yaxis.range = [20,100]
fig = go.Figure(data = [ema12,ema26], layout = layout)
iplot(fig)



In [7]:
# returns buy and sell signals
def signal(data):
  sigPriceBuy = []
  sigPriceSell = []
  flag = -1
  
  for i in range(len(data)):
    if (data["MACD"][i] > data['signal line'][i]) and (data['MACD'][i] < 0 and data['signal line'][i] < 0):
      if flag != 1:
        sigPriceBuy.append(stock['Close'][i])
        sigPriceSell.append(np.nan)
        flag = 1
      else:
        sigPriceSell.append(np.nan)
        sigPriceBuy.append(np.nan)
    elif (data["MACD"][i] < data['signal line'][i]) and (data['MACD'][i] > 0 and data['signal line'][i] > 0):
      if flag != 0:
        sigPriceSell.append(stock['Close'][i])
        sigPriceBuy.append(np.nan)
        flag = 0
      else:
        sigPriceSell.append(np.nan)
        sigPriceBuy.append(np.nan)
    else:
      sigPriceBuy.append(np.nan)
      sigPriceSell.append(np.nan)

  return (sigPriceBuy, sigPriceSell)



      

In [8]:
# storing signals in dataframe 

df = pd.DataFrame()
buyPrice = signal(data)[0]
sellPrice = signal(data)[1]

df['buy price'] = buyPrice
df['sell price'] = sellPrice

In [9]:
# MACD construction 
signalLine = go.Scatter(x = data.Date, y = data['signal line'], marker = {'color':'#ff8408'}, name = 'Signal Line')
macd = go.Scatter(x = data.Date, y = data['MACD'], marker = {'color':'MediumPurple'}, name = 'MACD Line')
negativeMomentum = go.Bar( x = data.Date, y = data.negative, marker = {'color':"red"}, name = "MACD Histogram negative momentum")
positiveMomentum = go.Bar( x = data.Date, y = data.positive, marker = {'color':"green"}, name = "MACD Histogram positive momentum")
layout = go.Layout(title = {'text':'Visualizing <b>MACD and Signal Line crossover</b>', 'x':0.5,'y':0.92, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.xaxis.range = ['2013-05-11','2014-03-11']
layout.yaxis.range = [-1.0,3]
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.yaxis.title.text = "AAPL Close Price"
layout.autosize = True
fig = go.Figure(data = [signalLine, macd, negativeMomentum, positiveMomentum], layout = layout)
fig.add_annotation(x="2013-08-26", y=2.361059, text="<i>Bearish Momentum</i> signaling a sell",arrowhead = 7, ax = 0, ay = -40)
fig.add_annotation(x="2013-09-25", y=0.00260, text="<i>Bullish Momentum</i> signaling a buy",arrowhead = 7, ax = 0, ay = 70, xanchor = "left")
fig.show()

# Buy / Sell signal visualization 
trace = go.Scatter(x = stock['Date'], y = stock['Close'], marker = {'color':"#3f8fcc"}, opacity = 0.5, name = "AAPL Close Price")
buy = go.Scatter(x = stock.Date, y = df['buy price'], marker = {'color':'green', 'symbol':'triangle-up','size':7}, name = "Buy", mode='markers')
sell = go.Scatter(x = stock.Date, y = df['sell price'], marker = {'color':'red', 'symbol':'triangle-down','size':7}, name = "Sell", mode = 'markers')
layout = go.Layout(title = {'text':'AAPL 10Y <b>Buy-Sell Signals</b>','x':0.5,'y':0.88,'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.yaxis.title.text = "AAPL close price"
fig = go.Figure(data = [trace,buy,sell], layout = layout)
fig.show()



In [10]:
# calculate profit and visualize it.
profit = df['sell price'].sum() - df['buy price'].sum()
print("AAPL stats : ")
print("----------------------------------------------------------------------")
print("Min : $" + str(stock['Close'].min()))
print("Max : $" + str(stock['Close'].max()))
print("Trading time : 10Y (June 03 2009  - June 03 2019) " )
print("----------------------------------------------------------------------")
print('Your profit on using MACD indicator stratergy : ' + str(profit))

AAPL stats : 
----------------------------------------------------------------------
Min : $17.06999969482422
Max : $232.07000732421875
Trading time : 10Y (June 03 2009  - June 03 2019) 
----------------------------------------------------------------------
Your profit on using MACD indicator stratergy : 90.23715591430664


In [11]:
# Accounting for profits and losses 
buy = [x for x in buyPrice if str(x) != 'nan']
sell = [x for x in sellPrice if str(x) != 'nan']
profit = []
loss = []
for i in range(len(buy)):
  difference = sell[i] - buy[i]
  if  difference > 0:
    profit.append(difference)
  elif difference < 0:
    loss.append(difference)


In [15]:
# Analysis of MACD trading stratergy
fig = go.Figure(go.Indicator(
    mode = "number+gauge",
    gauge = {'shape': "bullet",
             'bar':{'color':'green'}},
    value = df['sell price'].mean(),
    domain = {'x': [0.20, 1], 'y': [0.08, 0.25]},
    title = {'text': "Average value per trade"}))

fig.add_trace(go.Indicator(
    mode = "number+gauge",
    gauge = {'shape': "bullet",
             'bar':{'color':'red'}},
    value = df['buy price'].mean(),
    domain = {'x': [0.20, 1], 'y': [0.4, 0.6]},
    title = {'text': "Average cost per trade"}))

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",
             'bar':{'color':'orange'}},
    delta = {'reference': stock['Close'][len(stock) - 1] - 90},
    value = stock['Close'][len(stock) - 1],
    domain = {'x': [0.20, 1], 'y': [0.7, 0.9]},
    title = {'text': "Current value"}))

fig.show()



[-1.3928565979003906,
 -1.4200019836425781,
 -18.047138214111328,
 -2.8128509521484375,
 -10.709999084472656,
 -7.529998779296875,
 -10.300003051757812]

In [35]:
# Use `hole` to create a donut-like pie chart
labels = ['Profit','Loss']
values = []
fig = go.Figure(data=[go.Pie(labels=labels, values=[len(profit),len(loss)], hole=.3,title={'text':"Share of P/L in all trades",})])
fig.show()


In [36]:
# Visualizing using a box plot

trace_buy = go.Box(y = profit, name = "Profit")
trace_sell = go.Box(y = np.array(loss)*-1, name = "Loss")
layout = go.Layout(title={'text':"Visualizing <b>profit and loss distribution</b>", 'x':0.5,'y':1, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
fig = go.Figure(data = [trace_buy, trace_sell], layout = layout)
fig.show()


In [31]:
#using bubble plot 
x_profit = [i for i in range(len(profit))]
x_loss = np.array([i for i in range(len(profit), len(profit)+len(loss))])
trace1 = go.Scatter(x = x_profit, y = np.array(profit), marker={'size':np.array(profit) * 5}, mode="markers", name="profit")
trace2 = go.Scatter(x = x_loss, y = np.array(loss)*-1, marker={'size':np.array(loss)* -5}, mode="markers", name="loss")
layout = go.Layout(title={'text':"Visualizing <b>profit and loss distribution</b>", 'x':0.5,'y':0.92, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
fig = go.Figure(data = [trace1, trace2], layout = layout)
fig.show()

profit



[0.7585716247558594,
 1.4699993133544922,
 5.884286880493164,
 3.7814292907714844,
 6.391429901123047,
 4.6399993896484375,
 0.2100067138671875,
 9.684284210205078,
 8.159996032714844,
 12.159996032714844,
 16.68999481201172,
 7.2400054931640625,
 14.800003051757812,
 4.4199981689453125,
 12.44000244140625,
 13.779998779296875,
 7.9300079345703125,
 12.009994506835938]